# Summarizing Data Post QC

Author: Zan Koenig

## Index
1. [Setting Default Paths](#1.-Set-Default-Paths)
2. [Organizing the dataset](#2.-Setting-up-data)
3. [Annotating table with relatedness information](#3.-Annotating-table-with-relatedness-information)
4. [Calculating statistics per population](#4.-Calculating-statistics-per-population)
5. [Formatting table for exporting](#5.-Formatting-table-for-exporting)
6. [Exporting final table](#6.-Exporting-final-table)

# General Overview:

The purpose of this script is to format and write out a tsv which will be used to create plots and summaries of the post-QC dataset in R.

**This script contains information on how to:**
- Select specific columns from a matrix table
- Annotate filter flags onto a matrix table
- Join the columns of two matrix tables
- Join two tables
- Group a matrix table by region, population 
- Use `hl.agg.stats` to calculate statics for a metric within a population
- Count the number of samples where a filter flag equals True  



In [ ]:
import hail as hl

# 1. Set Default Paths
These default paths can be edited by users as needed. It is recommended to run these tutorials without writing out datasets. 

By default all of the write sections are shown as markdown cells. If you would like to write out your own datasets, you can copy the code and paste it into a new code cell. 

[Back to Index](#Index)

In [ ]:
# Input file 
post_qc_path = 'gs://hgdp-1kg/tutorial_datasets/metadata_and_qc/post_qc.mt'

# PCA outliers file 
outliers_path = 'gs://hgdp-1kg/tutorial_datasets/pca/pca_outliers.txt'

# Paths to related and unrelated Matrix Tables (without outliers) - written out in Notebook 2: PCA and Ancestry Analyses
unrelateds_path = 'gs://hgdp-1kg/tutorial_datasets/pca_results/unrelateds_without_outliers.mt'
relateds_path = 'gs://hgdp-1kg/tutorial_datasets/pca_results/relateds_without_outliers.mt'

# Path for final output table in tsv format
final_table_path = 'gs://hgdp-1kg/tutorial_datasets/metadata_and_qc/post_qc_summary.tsv'

# 2. Setting up data

Here we are walking through some steps to get the dataset ready for downstream analyses. We first create a table with only sample data, then select only the columns we need for the table we will write out. Next, we create a checkpoint of that table, to speed up downstream analysis steps.
 

<details><summary>For more information on Hail methods and expressions click <u><span style="color:blue">here</span></u>.</summary> 
<br>
<li><a href="https://hail.is/docs/0.2/utils/index.html#hail.utils.hadoop_open"> More on  <i> hadoop_open() </i></a></li>

<li><a href="https://hail.is/docs/0.2/methods/genetics.html#hail.methods.sample_qc"> More on  <i> sample_qc() </i></a></li>
    
<li><a href="https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.cols"> More on  <i> cols() </i></a></li>
    
<li><a href="https://hail.is/docs/0.2/hail.Table.html#hail.Table.select"> More on  <i> select() </i></a></li>    
    
<li><a href="https://hail.is/docs/0.2/methods/impex.html#hail.methods.read_table"> More on  <i> read_table() </i></a></li>

<li><a href="https://hail.is/docs/0.2/hail.Table.html#hail.Table.count"> More on  <i> count() </i></a></li>
    
<li><a href="https://hail.is/docs/0.2/hail.Table.html#hail.Table.checkpoint"> More on  <i> checkpoint() </i></a></li>
     
</details>

[Back to Index](#Index)

In [ ]:
# Reading in a version of the dataset which has gnomAD's variant and sample QC filters applied to it
mt = hl.read_matrix_table(post_qc_path)

In [ ]:
# Removing PCA outliers from the dataset
# To read in the PCA outlier list, first need to read the file in as a list
# Using hl.hadoop_open here which allows one to read in files into Hail from Google Cloud Storage
with hl.utils.hadoop_open(outliers_path) as file:
    outliers = [line.rstrip('\n') for line in file]

# Using hl.literal here to convert the list from a python object to a hail expression so that it can be used to filter out samples
outliers_list = hl.literal(outliers)

# Using the list of PCA outliers, using the ~ operator which is a negation operator and obtains the compliment
# In this case the compliment is samples which are not contained in the pca outlier list
mt_without_outliers = mt.filter_cols(~outliers_list.contains(mt['s']))

In [ ]:
mt_without_outliers.count() 

In [ ]:
# Grabbing only the columns from the Matrix Table (outputs table of just columns)
mt_col_table = mt_without_outliers.cols()

In [ ]:
# Writing a col table with only the columns needed for table 1
mt_col_table = mt_col_table.select(mt_col_table.hgdp_tgp_meta.genetic_region,
                             mt_col_table.hgdp_tgp_meta.population,
                             mt_col_table.sample_qc.n_snp, 
                             mt_col_table.bam_metrics.mean_coverage)

In [ ]:
# Validity check - there should be 4096 samples 
mt_col_table.count()

In [ ]:
## Writing out col_table as a checkpoint to make the downstream steps run faster
## This is done because running sample_qc is computationally expensive
#col_table.checkpoint(checkpoint_path, overwrite=True)

In [ ]:
# This is a table of only the columns with only postQC information
col_table = hl.read_table(checkpoint_path)

In [ ]:
# Since col_table is a table, count prints the number of rows which is equal to the number of samples
# There should be 4096 samples
mt_col_table.count()

# 3. Annotating table with relatedness information

Relatedness information is added to the dataset so that we can filter out related individuals. 

<details><summary>For more information on Hail methods and expressions click <u><span style="color:blue">here</span></u>.</summary> 
    
<br>
<li><a href="https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.annotate_cols"> More on  <i> annotate_cols() </i></a></li>

<li><a href="https://hail.is/docs/0.2/aggregators.html#hail.expr.aggregators.counter"> More on  <i> counter() </i></a></li>
    
<li><a href="https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.union_cols"> More on  <i> union_cols() </i></a></li>
    
</details>

[Back to Index](#Index)

In [ ]:
# Need to get number of unrelateds annotated to the table

# Reading in the unrelated and related Matrix Tables which were written out in notebook 2: PCA and Ancestry Analyses
unrelateds = hl.read_matrix_table(unrelateds_path)
relateds = hl.read_matrix_table(relateds_path)

# Annotating both the unrelated and the related tables with a flag named unrelated 
# Set unrelated flag to True for those in the unrelated dataset, and False for those in the related dataset
unrelateds = unrelateds.annotate_cols(unrelated = True)
relateds = relateds.annotate_cols(unrelated = False)

# Using hl.cols() to obtain two tables with only the columns from the original Matrix Tables
unrelateds_cols = unrelateds.cols()
relateds_cols = relateds.cols()

In [ ]:
# Validity check 
print(unrelateds_cols.count(), relateds_cols.count()) # 3378 unrelated and 718 related samples = 4096 total samples 

In [ ]:
# Annotating the unrelated/related mts with counts per population
unrelateds_count = unrelateds_cols.aggregate(hl.agg.counter(unrelateds_cols.hgdp_tgp_meta.population))
relateds_count = relateds_cols.aggregate(hl.agg.counter(relateds_cols.hgdp_tgp_meta.population))

# Validity check - print out the number of unrelated and related individuals per population 
print(f"Number of unrelated individuals per population: \
{unrelateds_count}\n\nNumber of related individuals per population: {relateds_count}")

In [ ]:
# Joining the columns of the unrelated and related datasets
mt_combined = unrelateds.union_cols(relateds)

# Validity check - count the number of unrelateds (True values) in the Matrix Table to make sure it is as expected
print(mt_combined.aggregate_cols(hl.agg.counter(mt_combined.unrelated))) # 3378 True and 718 False

# Creating a table with only the columns from the Matrix Table containing related information
# This is done since the final output will be a tsv and thus must be in table format
# Being a table of columns allows it to be annotated onto the existing mt_col_table as shown below
mt_combined_col_table = mt_combined.cols()

# Annotating the relatedness information onto the column table
mt_col_table = mt_col_table.annotate(unrelated = mt_combined_col_table[mt_col_table.s].unrelated)

In [ ]:
mt_col_table.show(5)

# 4. Calculating statistics per population
In this section, we will be using `hl.agg.stats()` which calculates the following metrics for a given expression:
- min
- max
- mean
- standard deviation
- number of non-missing records
- sum
 
Using `hl.group_by()` we calculate these statistics for each of the 78 populations in this dataset.
We also count the number of related samples within each populations by using `hl.agg.count_where()` and counting the number of times the field denoting if samples are related or not is True.

<br>
<details><summary>For more information on Hail methods and expressions click <u><span style="color:blue">here</span></u>.</summary> 
<br>
<li><a href="https://hail.is/docs/0.2/aggregators.html#hail.expr.aggregators.stats"> More on  <i> stats() </i></a></li>

<li><a href="https://hail.is/docs/0.2/hail.Table.html#hail.Table.group_by"> More on  <i> group_by() </i></a></li>
    
<li><a href="https://hail.is/docs/0.2/aggregators.html#hail.expr.aggregators.count_where"> More on  <i> count_where() </i></a></li>
    
<li><a href="https://hail.is/docs/0.2/hail.expr.TupleExpression.html#hail.expr.TupleExpression.show"> More on <i>show()</i></a></li>
     
</details>

[Back to Index](#Index)

In [ ]:
# Calculating stats per population for each metric grouped by genetic region and population
table = n_snp = mt_col_table.group_by(
    mt_col_table.genetic_region, mt_col_table.population).aggregate(
    n_snp_stats = hl.agg.stats(mt_col_table.n_snp),
    cov_stats = hl.agg.stats(mt_col_table.mean_coverage),
    n_unrelated = hl.agg.count_where(mt_col_table.unrelated == True))

In [ ]:
# Checking that each of the table fields contain what we'd expect
table.show()

# 5. Formatting table for exporting
In this section, we format the table before exporting so it is in a usable format once written out. Specifically, we are flattening the table. This is done so that when the table is written out, the data is easier to work with. If the tables were written out without flattening them, the new annotated information would be in a nested structure which would make it difficult to work with outside hail. 

<br>
<details><summary>For more information on Hail methods and expressions click <u><span style="color:blue">here</span></u>.</summary> 
    
<br>
<li><a href="https://hail.is/docs/0.2/hail.Table.html#hail.Table.flatten"> More on  <i> flatten() </i></a></li>
    
<li><a href="https://hail.is/docs/0.2/hail.Table.html#hail.Table.key_by"> More on <i>key_by()</i></a></li>
    
<li><a href="https://hail.is/docs/0.2/hail.Table.html#hail.Table.describe"> More on <i>describe()</i></a></li>
    
</details>

[Back to Index](#Index)

In [ ]:
# Flattening out the structs created from annotating the tables
table = table.flatten()

# Changing the keys of the table so that it is keyed by genetic region and population
table = table.key_by(table.genetic_region, table.population)

In [ ]:
# Checking format of the flattened table
table.show()

In [ ]:
# Checking on the format of the table after flattening to make sure it is what we'd expect
table.describe()

In [ ]:
# One last validity check before writing out the dataset to make sure you still have the number of rows you expect
# In this case, since the data is grouped by genetic region, population
# The number of rows should be equal to the number of populations (78)
table.count()

# 6. Exporting final table
<br>
<details><summary>For more information on Hail methods and expressions click <u><span style="color:blue">here</span></u>.</summary> 
<br>
<li><a href="https://hail.is/docs/0.2/hail.Table.html#hail.Table.export"> More on  <i> export() </i></a></li>

</details>

[Back to Index](#Index)

In [ ]:
# Writing out the final table in tsv format 
table.export(final_table_path, header=True)

[Back to Index](#Index)